In [1]:
!source activate /home/gilbee3/anaconda3/envs/akita

In [2]:
BASE_PATH = "/".join(os.getcwd().split("/")) # base directory level

BIN_PATH = os.path.join(BASE_PATH, "bin")  # where my scripts live
DATA_PATH = os.path.join(BASE_PATH, "data")  # where I dump new data.
RESULTS_PATH = os.path.join(BASE_PATH, "results")  # where I analyze results



# Functions

In [ ]:
def dictionary_of_baseline_comparison_dfs(list_of_indivs, baseline):
    comp_dict = {}

    for ind in list_of_indivs:
        filename = '3dcomp_%s_vs_%s.txt' % (ind, baseline)
        filepath = '%s/%s' % (COMP_PATH,filename)
        if os.path.exists(filepath):
            df = pd.read_table(filepath)
            df = df[df.chr != 'chrX']
            comp_dict[ind] = df
    
    return df

def create_genomewide_and_windowwise_summary_dfs(comp_dict, windows_to_keep):
    indivs = comp_dict.keys()
    window_divergence = pd.DataFrame(index = windows_to_keep, columns = indivs)
    genome_wide = pd.DataFrame(index = indivs, columns = ['genome_avg_divergence', 'genome_avg_spearman','genome_avg_mse'])

    genome_wide['temp'] = genome_wide.index
    new = genome_wide.temp.str.split("_", expand = True)
    genome_wide['super_pop'] = new[0]
    genome_wide['sub_pop'] = new[1]
    genome_wide['sex'] = new[2]
    genome_wide['id'] = new[3]
    genome_wide = genome_wide.drop(columns=['temp'])

    for ind in indivs:
        df = comp_dict[ind]

        avg_mse =float(np.mean(df['mse']))
        avg_spearman = float(np.mean(df['spearman']))
        avg_divergence = 1 - avg_spearman

        genome_wide['genome_avg_mse'].loc[ind] = avg_mse
        genome_wide['genome_avg_spearman'].loc[ind] = avg_spearman
        genome_wide['genome_avg_divergence'].loc[ind] = avg_divergence

        df = df.set_index(['chr','windowStartPos'])

        window_divergence[ind] = 1 - df['spearman']

    window_divergence.columns = window_divergence.columns.str.split('_')

    return genome_wide, window_divergence


# 1KG vs Ancestral 

## 3D Predictions

## Sequence Comparisons